In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
import pandas as pd

from massbank2db.db import MassbankDB

# Tutorial: Build a local copy of MassBank

This tutorial illustrates how to use this package to build a local MassBank database (DB) using SQLite. 

## Preparation 

- Download the MassBank data, e.g. by cloning the [MassBank repository](https://github.com/MassBank/MassBank-data) from GitHub. 
- Build a local copy of PubChem using the [pubchem2sqlite package](https://github.com/bachi55/local_pubchem_db) **[optional]**
  - to homogenouse the Massbank structure information using PubChem
  - to retrieve molecular candidate sets for the MassBank spectra

**Please note:** This tutorial uses a [fork of Massbank](https://github.com/bachi55/MassBank-data/tree/fix_entry_format) where minor formatting issues have been fixed.

## General Concept

MassBank organizes their datasets by contributors (sub-directories in the MassBank root-directory) and each contributor additionally into accession prefixes, such as EA, KW, etc. In the ```massbank2db``` package, each (contributor, accession-prefix)-tuple is considered to be a separate dataset. That means, MassBank entries (or records) are added in chunks encompassing all spectra of a certain (contributor, accession-prefix)-tuple.

Furthermore, each chunk is divided into sub-datasets based on the mass-spectrometry (MS) (including the ionization mode / polarity) and (liquid-)chromatographic (LC) configuration. Ultimately, we add spectra grouped as different datasets using the following scheme: 
```
Example:
    (AU, Athens_Univ)
        -> AU_000  ~ positive, LC configuration 1, MS instrument 1
        -> AU_001  ~ positive, LC configuration 2, MS instrument 1
        -> AU_002  ~ negative, LC configuration 3, MS instrument 1
        -> AU_003  ~ negative, LC configuration 4, MS instrument 2
        -> ... 
    ...
    (PR, RIKEN)
        -> PR_000  ~ positive, LC configuration 1, MS instrument 1
        -> PR_001  ~ negative, LC configuration 1, MS instrument 1
    ...
```
The motivation behind this approach is, that ```massbank2db``` originates from a project where MS/MS-spectrum *and* retention time (RT) information was supposed to be used to *jointly* train a machine learning approach for small molecule identification. As RTs are not (directly) compareable between LC configurations, the spectra where grouped such that RT compareability was maximized within one dataset. The following SQLite statement was used to group MassBank entries associated with an (contributor accession-prefix)-tuple:
```sql
GROUP BY ion_mode, instrument, instrument_type, column_name, column_temperature, flow_gradient, flow_rate, solvent_A, solvent_B
```

However, nothing prevents the user from loading all spectra and RTs without considering the grouping imposed by the dataset.

## Building the Database

First we get the list (contributors, accession-prefix)-tuples:

In [12]:
massbank_dir = "/run/media/bach/EVO500GB/data/MassBank"
mbds = pd.read_csv(os.path.join(massbank_dir, "List_of_Contributors_Prefixes_and_Projects.md"),
                   sep="|", skiprows=2, header=None) \
    .iloc[:, [1, 4]] \
    .applymap(str.strip) \
    .rename({1: "Contributor", 4: "AccPref"}, axis=1)  # type: pd.DataFrame

mbds.head()

,Contributor,AccPref
0,ACES_SU,AS
1,AAFC,AC
2,Athens_Univ,AU
3,BGC_Munich,RP
4,Boise_State_Univ,BSU


Than we initialize the MassBank DB tables:

In [14]:
mb_dbfn = "massbank_example.sqlite"
with MassbankDB(mb_dbfn) as mbdb:
    mbdb.initialize_tables(reset=True)

Now we can insert the MassBank entries into the DB. We are using three options of the ```insert_dataset``` function, that will be further explained:
- ```use_pubchem_structure_info```: If True, the PubChem CIDs (if available) or the InChIKeys provided in the MassBank entries are used to query the compound structure information from PubChem and replace the information from MassBank with the ones from PubChem. In this way, we can ensure all SMILES structure representations have been determined using the same procedure (by PubChem). This is particularly relevant, if molecular descriptors or fingerprints should be calculated from the structures. (See also: https://jcheminf.biomedcentral.com/articles/10.1186/s13321-018-0293-8)
- ```only_ms2```: If True, the DB will only contain MS/MS spectra (determined by ```AC$MASS_SPECTROMETRY: MS_TYPE MS2```). TODO: Currently, this also filters isotope pattern spectra, which might be useful in some applications. 
- ```only_with_rt```: If True, only spectra with retention time information are included in the DB.

In [15]:
# Filename of the local PubChem DB
pc_dbfn = "/run/media/bach/EVO500GB/data/pubchem_24-06-2019/db/pubchem.sqlite"

with MassbankDB(mb_dbfn) as mbdb:
    for idx, row in mbds.iterrows():
        print("(%02d/%02d) %s: " % (idx + 1, len(mbds), row["Contributor"]))
        for pref in map(str.strip, row["AccPref"].split(",")):
            print(pref)
            mbdb.insert_dataset(pref, row["Contributor"], massbank_dir, 
                                pc_dbfn=pc_dbfn,
                                use_pubchem_structure_info=True,
                                only_ms2=True,
                                only_with_rt=True)

(01/48) ACES_SU: 
AS
(02/48) AAFC: 
AC
(03/48) Athens_Univ: 
AU
(04/48) BGC_Munich: 
RP
(05/48) Boise_State_Univ: 
BSU
(06/48) BS: 
BS


[WARNING] massbank2db.spectrum : (BS001075) Multiple compounds (n=2) matching the InChIKey (IPCSVZSSVZVIGE-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (BS001075) Multiple compounds (n=2) matching the InChIKey (IPCSVZSSVZVIGE-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (BS001077) Multiple compounds (n=2) matching the InChIKey (IPCSVZSSVZVIGE-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (BS001077) Multiple compounds (n=2) matching the InChIKey (IPCSVZSSVZVIGE-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (BS003871) Multiple compounds (n=2) matching the InChIKey (GHVNFZFCNZKVNT-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (B

(07/48) CASMI2012: 
SMI
(08/48) CASMI2016: 
SM
(09/48) Chubu_Univ: 
UT


[WARNING] massbank2db.spectrum : (UT001356) Multiple compounds (n=2) matching the InChIKey (SLKDGVPOSSLUAI-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (UT001356) Multiple compounds (n=2) matching the InChIKey (SLKDGVPOSSLUAI-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (UT001860) Multiple compounds (n=2) matching the InChIKey (LVNGJLRDBYCPGB-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (UT001860) Multiple compounds (n=2) matching the InChIKey (LVNGJLRDBYCPGB-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (UT001122) Multiple compounds (n=2) matching the InChIKey (LVNGJLRDBYCPGB-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (U

(10/48) Eawag: 
EA
EQ
(11/48) Eawag_Additional_Specs: 
ET
ETS
(12/48) Env_Anal_Chem_U_Tuebingen: 
TUE
(13/48) Fac_Eng_Univ_Tokyo: 
JP
(14/48) Fiocruz: 
FIO
(15/48) Fukuyama_Univ: 
FU
(16/48) GL_Sciences_Inc: 
GLS
(17/48) HBM4EU: 
HB
(18/48) IPB_Halle: 
PB
PN
(19/48) ISAS_Dortmund: 
IA
(20/48) JEOL_Ltd: 
JEL
(21/48) Kazusa: 
KZ
(22/48) Keio_Univ: 
KO
(23/48) KWR: 
KW
(24/48) Kyoto_Univ: 
CA
(25/48) LCSB: 
LU
(26/48) Literature_Specs: 
LIT
(27/48) MetaboLights: 
ML
(28/48) Metabolon: 
MT
(29/48) MPI_for_Chemical_Ecology: 
CE
(30/48) MSSJ: 
MSJ
(31/48) NAIST: 
KNA


[ERROR] massbank2db.db : (KNA00615) SQLite integrity error: NOT NULL constraint failed: spectra_meta.precursor_type
[ERROR] massbank2db.db : (KNA00615) SQLite integrity error: NOT NULL constraint failed: spectra_meta.precursor_type
[ERROR] massbank2db.db : (KNA00533) SQLite integrity error: NOT NULL constraint failed: spectra_meta.precursor_type
[ERROR] massbank2db.db : (KNA00533) SQLite integrity error: NOT NULL constraint failed: spectra_meta.precursor_type
[WARNING] massbank2db.spectrum : (KNA00526) Multiple compounds (n=2) matching the InChIKey (KDYFGRWQOYBRFD-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (KNA00526) Multiple compounds (n=2) matching the InChIKey (KDYFGRWQOYBRFD-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[ERROR] massbank2db.db : (KNA00526) SQLite integrity error: NOT NULL constraint failed: spectra_meta.precursor_type
[ERROR] massbank2db.db : (KNA00526

(32/48) NaToxAq: 
NA
(33/48) Nihon_Univ: 
NU
(34/48) Osaka_MCHRI: 
MCH
(35/48) Osaka_Univ: 
OUF
(36/48) PFOS_research_group: 
FFF
(37/48) RIKEN: 
PR


[WARNING] massbank2db.spectrum : (PR305109) Multiple compounds (n=2) matching the InChIKey (AADVZSXPNRLYLV-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (PR305109) Multiple compounds (n=2) matching the InChIKey (AADVZSXPNRLYLV-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (PR304976) Multiple compounds (n=2) matching the InChIKey (CLDCTFPNFRITPI-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (PR304976) Multiple compounds (n=2) matching the InChIKey (CLDCTFPNFRITPI-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (PR304747) Multiple compounds (n=2) matching the InChIKey (INYYVPJSBIVGPH-UHFFFAOYSA-N). Taking the one with the lowest CID to update the compound information.
[WARNING] massbank2db.spectrum : (P

(38/48) RIKEN_IMS: 
LQA
LQB
(39/48) RIKEN_NPDepo: 
NGA
CB
(40/48) RIKEN_ReSpect: 
PT
PS
PM
(41/48) Tottori_Univ: 
TT
(42/48) UFZ: 
UF
UA
UN
UP
(43/48) Univ_Connecticut: 
CO
(44/48) Univ_Toyama: 
TY
(45/48) UOEH: 
UO
(46/48) UPAO: 
UPA
(47/48) Washington_State_Univ: 
BML
(48/48) Waters: 
WA
